<a href="https://colab.research.google.com/github/blueocean3D/AITradingBot/blob/main/Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas==1.1.5 numpy==1.19.4 scipy==1.5.4

In [ ]:
#BUILD CURRENT DAY POSITIONS DATA TABLE
import pandas as pd
import requests
import json
import time
import numpy as np
from pandas.io.json import json_normalize
def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

data = requests.get('https://financialmodelingprep.com/api/v3/stock-screener?exchange=NASDAQ,nyse,amex&apikey=)


d = data.json()
ticker = json_extract(d, 'symbol')



def pct_change(first, second):
    diff = second - first
    change = 0
    try:
      
      if diff > 0:
            change = (diff / first) * 100
      elif diff < 0:
            diff = first - second
            change = -((diff / first) * 100)
    except ZeroDivisionError:
      return float('inf')
    return change
inputs = []
x4 = []
df3 = pd.DataFrame()
df4 = pd.DataFrame()
for value in ticker:
    try:
    
      response = requests.get('https://financialmodelingprep.com/api/v3/discounted-cash-flow/' + (value) + '?apikey=')
    
        
      dcf_value = response.json()
      dcf = json_extract(dcf_value, 'dcf')
      symbol = json_extract(dcf_value, 'symbol')
      
      closeprice = json_extract(dcf_value, 'Stock Price')
   
      
      row2=pd.Series(dcf)
      
      row3=pd.Series(closeprice)
      row4=pd.Series(symbol)
      
      x1 = np.array(dcf,dtype=np.float)
      x2 = np.array(closeprice,dtype=np.float)
      x3 = x1/x2
      row6 = pd.Series(x3)
      d3 = {"symbol":row4, "dcf": row2, "closeprice": row3, "change": row6}
      df3 = pd.DataFrame(d3)
      df4 = df4.append(df3)
             
    except Exception as e:
      raise e


      
df4


In [3]:
#FILTERING, ANALYSIS AND BEST DAILY POSITIONS TO TAKE
import pandas as pd
#x3 = pct_change (x1,x2)       
#inputs.append(x3)

#import alpaca_trade_api as tradeapi
def buy_sell(x):
    if x < 1:
        return 'sell'

    else:
        return 'buy'

def normalize(z):
    if z < 1:
      return z * -1
    else: 
      return  z * 1
      


df4 = df4.sort_values(by = 'change')


df4 = df4.replace([np.inf, -np.inf], np.nan).dropna(subset=['change'], how="all")
df4= df4[(df4.change > 0) | (df4.change < 0)]
df4 = df4[(df4.dcf > 0)].dropna()
df4 = df4.head(5).append(df4.tail(5))


#WEIGHTS CALCULATION
sum_column = df4.sum(axis = 0)

df4['weights'] = df4['change'] /  df4['change'].sum()

df4['Cash Allocation'] = df4['weights'] * 1000 # Portfolio Investment Amount

df4 ['Volume'] = df4['Cash Allocation']/df4['closeprice']

df4 ['Round_Volume'] = df4['Volume'].apply(np.ceil)
 



df4['Buy/Sell'] = df4['change'].apply(lambda x: buy_sell(x))
df4 ['Normalize'] = df4['Round_Volume'].apply(lambda z: normalize(z))
df4 = df4.replace(0, pd.np.nan).dropna(axis=0, how='any').fillna(0)





         
  
df4


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


,symbol,dcf,closeprice,change,weights,Cash Allocation,Volume,Round_Volume,Buy/Sell,Normalize
0,OCSI,2.799278e-01,8.51,0.032894,1.428248e-07,0.000143,0.000017,1.0,sell,1.0
0,EFC,7.356458e-01,18.55,0.039657,1.721916e-07,0.000172,0.000009,1.0,sell,1.0
0,FOR,7.662500e-01,19.26,0.039785,1.727434e-07,0.000173,0.000009,1.0,sell,1.0
0,ARL,5.681654e-01,13.40,0.042400,1.841014e-07,0.000184,0.000014,1.0,sell,1.0
0,ARCC,1.157187e+00,20.63,0.056092,2.435518e-07,0.000244,0.000012,1.0,sell,1.0
0,AVAL,3.054817e+04,5.87,5204.117856,2.259614e-02,22.596140,3.849428,4.0,buy,4.0
0,CIB,2.125389e+05,34.99,6074.275203,2.637434e-02,26.374340,0.753768,1.0,buy,1.0
0,UNG,3.429623e+05,19.28,17788.500000,7.723719e-02,77.237191,4.006078,5.0,buy,5.0
0,MKGI,2.243160e+05,2.27,98817.638586,4.290635e-01,429.063543,189.014776,190.0,buy,190.0
0,USCI,4.243480e+06,41.43,102425.290369,4.447279e-01,444.727870,10.734441,11.0,buy,11.0
